In [ ]:
### DEMARRER ICI SI AUCUN DATASET DE QUESTIONS DISPO ###
### SINON VOIR PLUS LOIN ###

import openpyxl
import polars as pl

print("load")
wb = openpyxl.load_workbook('data.xlsx')
print("loaded")
ws = wb.active

label = [str(ws.cell(row=i,column=1).value) for i in range(1+1,ws.max_row+1)]
titre = [ws.cell(row=i,column=2).value for i in range(1+1,ws.max_row+1)]
texte = [ws.cell(row=i,column=3).value for i in range(1+1,ws.max_row+1)]

ids = list(range(0,len(label)))


print(label)
print(titre)
print(texte)
df = pl.DataFrame(
    {   "label" : label,
        "titre" : titre,
        "texte": texte,
        "id" : ids
    }
)

print(df)
print(df[0,1])
print(len(df))

id2label = dict(zip(ids, label))
label2id = dict(zip(label, ids))

print(id2label)
print(label2id)

In [ ]:
import ollama

df_questions = pl.DataFrame(
    {   "id" : [],
        "question" : [],
    },
    schema=[("id", pl.Int64), ("question", pl.String)]
    )

n = len(df)
model='wizardlm2:7b'

for i in range(0, n):

    prompt = """Tu aides à la création d'un jeu d'enquête nommé "Dieu est mort".
    Le principe du jeu est le suivant : Dieu est mort, quatre archanges et cinq princes-démons figurent sur la liste des suspects.
    Chaque joueur incarne un archange ou un prince-démon. Les joueurs doivent découvrir ce qui est arrivé à Dieu, en discutant entre eux et en obtenant des indices. 
    Sans mentionner l'indice, crée 20 possibles questions à poser au maître du jeu pour obtenir l'indice suivant : """ + str(df[i,1]) + " : " + str(df[i,2])

    generated = ollama.generate(model=model, #choisir un modèle
                            prompt=prompt)

    id = str(df[i,3])
    res = generated["response"]
    res_list = res.split("\n")

    id_arr = [int(id)] * len(res_list)
    print(id_arr)

    df_temp = pl.DataFrame(
    {   "id" : id_arr,
        "question" : res_list,
    }
    )

    df_questions = pl.concat(
        [df_questions, df_temp]
    )

print(df_questions)

In [ ]:
import polars as pl

path = "questions_" + model + ".csv"
                    
df_questions.write_csv(path,
                       separator=";")

In [ ]:
# POUR GENERER DES QUESTIONS SUPPLEMENTAIRES POUR UN INDICE SPECIFIQUE

import ollama

df_questions = pl.DataFrame(
    {   "id" : [],
        "question" : [],
    },
    schema=[("id", pl.Int64), ("question", pl.String)]
    )

i = 5
model='wizardlm2:7b' # choisir modèle

prompt = """Tu aides à la création d'un jeu d'enquête nommé "Dieu est mort".
Le principe du jeu est le suivant : Dieu est mort, quatre archanges et cinq princes-démons figurent sur la liste des suspects.
Chaque joueur incarne un archange ou un prince-démon. Les joueurs doivent découvrir ce qui est arrivé à Dieu, en discutant entre eux et en obtenant des indices. 
Sans mentionner l'indice, crée 5 possibles questions à poser au maître du jeu pour obtenir l'indice suivant : """ + str(df[i,1]) + " : " + str(df[i,2])

print(prompt)

generated = ollama.generate(model=model,
                        prompt=prompt)

id = str(df[i,3])
res = generated["response"]
res_list = res.split("\n")

id_arr = [int(id)] * len(res_list)
print(id_arr)

df_temp = pl.DataFrame(
{   "id" : id_arr,
    "question" : res_list,
}
)

df_questions = pl.concat(
    [df_questions, df_temp]
)

print(df_questions)

path = "questions_" + str(i) + "_" + model + ".csv"
                    
df_questions.write_csv(path,
                       separator=";")

NOTES

Modèles utilisés pour la génération de questions :
- Openhermes 2.5 7B
- Gemma 7B
- Gemma 2 9B
- Gemma 2 27B
- Mistral-Nemo
- Orca2 13B
- WizardLM2 7B


Nettoyage : 
- Assembler les multiples csv en un seul nommé "questions_all.csv". Nettoyage manuel possiblement nécessaire (supprimer numéros de questions, signes et lignes inutiles, espaces en trop...)

In [ ]:
### DEMARRER ICI SI DATASET DE QUESTIONS DEJA GENERE ###

import polars as pl 
df_questions = pl.read_csv(source="questions_all.csv", 
                           has_header=True,
                           separator= ';')
df_questions.columns = ["label", "text"]

print(df_questions)

num_labels = len(df_questions["label"].unique())
print(num_labels)

print(df_questions["label"].unique().sort())


In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


train, test = train_test_split(df_questions, test_size=0.1)

train, eval = train_test_split(train, test_size=0.1)

eval_db = eval["text"]

x = []
for i in range(len(eval_db)):
    y = eval_db[i][:512]
    x.append(y)
eval_db = x

data_train = Dataset.from_polars(train,
                           split="train")

data_test = Dataset.from_polars(test,
                           split="test")

print(data_train)
print(data_train["label"])
print(data_train["text"])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./distilbert-base-multilingual-cased")

def preprocess_function(database):
    return tokenizer(database["text"], 
                     truncation=True
                     )


tokenized_train = data_train.map(preprocess_function, batched=True)
tokenized_test = data_test.map(preprocess_function, batched=True)

print(tokenized_train)

print(tokenized_train["text"])
print(tokenized_train["label"])
print(tokenized_train["input_ids"])
print(tokenized_train["attention_mask"])



In [ ]:
id2titre = dict(zip(ids, titre))
titre2id = dict(zip(titre, ids))
print(id2titre)
print(titre2id)

print(id2label)
print(label2id)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model = AutoModelForSequenceClassification.from_pretrained(
    "./distilbert-base-multilingual-cased",
    num_labels = num_labels,
    id2label=id2titre, 
    label2id=titre2id)

training_args = TrainingArguments(

    output_dir="model_classif",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False

)

trainer = Trainer(

    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                      model = "./model_classif/checkpoint-2576/", 
                      device = "cuda")

result = classifier("Que s'est-il passé en yougoslavie ?")
print(result)

In [ ]:
from transformers import pipeline
import polars as pl
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

classifier = pipeline("text-classification", 
                      model = "./model_classif/checkpoint-2576/", 
                      device = "cuda")

result = classifier(eval_db)
print(result)
pred = []

for i in range(len(result)):
    x = result[i]["label"]
    pred.append(x)

eval = pl.DataFrame(eval)
eval = eval.select("label").cast(pl.String)
true = eval.with_columns(label = pl.col("label").replace(id2titre))

df_results = pl.DataFrame({
    "true":true,
    "pred":pred

})

conf = confusion_matrix(true, pred)

accuracy = accuracy_score(true, pred)

print(accuracy)

report = classification_report(true, pred)

print(report)